# 김기범 전치리 코드 테스트

In [2]:
import pandas as pd

In [3]:
train_df = pd.read_csv('../data/train.csv')

train_df.shape

(52530, 8)

# 1. add predefined news category

In [4]:
train_df['input_text'][:10]

0         유튜브 내달 2일까지 크리에이터 지원 공간 운영
1            어버이날 막따가 흐려저…남부지방 여튼 황사
2            어버이날 맑다가 흐려져…남부지방 옅은 황사
3        내년부터 국가RD 평가 때 논문건수는 반영 않는다
4    김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것
5     회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간
6     회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간
7         야외서 생방송 하세요…액션캠 전용 요금제 잇따라
8         월드컵 태극전사 16강 전초기지 레오강 입성종합
9                         미세먼지 속 출근길
Name: input_text, dtype: object

In [5]:
train_df['input_text'][:10] + " [SEP] " + train_df['predefined_news_category'][:10]

0         유튜브 내달 2일까지 크리에이터 지원 공간 운영 [SEP] IT과학
1            어버이날 막따가 흐려저…남부지방 여튼 황사 [SEP] 생활문화
2            어버이날 맑다가 흐려져…남부지방 옅은 황사 [SEP] 생활문화
3        내년부터 국가RD 평가 때 논문건수는 반영 않는다 [SEP] IT과학
4    김명자 신임 과총 회장 원로와 젊은 과학자 지혜 모을 것 [SEP] IT과학
5     회새긴간 작까 김동시 걍심꼬백 뜽 새 소설집 뚜권 출간 [SEP] 생활문화
6     회색인간 작가 김동식 양심고백 등 새 소설집 2권 출간 [SEP] 생활문화
7         야외서 생방송 하세요…액션캠 전용 요금제 잇따라 [SEP] IT과학
8          월드컵 태극전사 16강 전초기지 레오강 입성종합 [SEP] 스포츠
9                           미세먼지 속 출근길 [SEP] 사회
dtype: object

# 2. AI HUB 뉴스 기사 데이터 가져오기

In [6]:
from glob import glob

In [7]:
ai_hub_017 = glob('../data/ai_hub_017/*.json')
ai_hub_017

['../data/ai_hub_017/TS_unanswerable.json',
 '../data/ai_hub_017/TS_span_inference.json',
 '../data/ai_hub_017/VS_unanswerable.json',
 '../data/ai_hub_017/VS_span_inference.json',
 '../data/ai_hub_017/TS_text_entailment.json',
 '../data/ai_hub_017/VS_span_extraction.json',
 '../data/ai_hub_017/TS_span_extraction.json',
 '../data/ai_hub_017/VS_text_entailment.json']

In [8]:
import json

df = pd.DataFrame()

for path in ai_hub_017:
    try:
        with open(path, 'r') as f:
            js = json.loads(f.read())
    except Exception as e:
        print(e)

    new_df = pd.json_normalize(js['data'])

    df = pd.concat([df, new_df], axis=0)

df.shape

Invalid control character at: line 250348 column 279 (char 18754979)


(158020, 8)

In [9]:
df.columns

Index(['doc_id', 'doc_title', 'doc_source', 'doc_published', 'created',
       'paragraphs', 'doc_class.class', 'doc_class.code'],
      dtype='object')

In [10]:
df.head()

,doc_id,doc_title,doc_source,doc_published,created,paragraphs,doc_class.class,doc_class.code
0,01100201.20210225173715002,"“시진핑엔 선물, 바이든엔 모욕”… 유럽의회서 막힌 中·EU투자협정",국민일보,20210225,20211217144411,[{'context': '중국과 유럽연합(EU)이 체결하기로 합의한 포괄적투자협정(...,None,국제
1,01100201.20210225092831001,김경수 분노…“내가 문 대통령께 반발? 참 희한한 일”,국민일보,20210225,20211217144411,[{'context': '문재인 대통령의 최측근으로 꼽히는 김경수 경남지사는 25일...,None,정치
2,01100201.20210225111004001,오늘부터 사라진 네이버 ‘실검’…빈자리엔 날씨와 주가,국민일보,20210225,20211217144411,[{'context': '국내 최대 포털 네이버에서 25일 ‘실시간 급상승 검색어(...,None,IT과학
3,01100201.20210225040953001,"경남도, 신혼 반값 임대주택 공급 확대",국민일보,20210225,20211217144411,"[{'context': '경남도와 김해시, 경남개발공사, 인제대학교가 청년·신혼부부...",None,사회
4,01100201.20210225161345001,작년 3분기 일자리 늘었다는데… 2030 일자리만 15만개 감소,국민일보,20210225,20211217144411,[{'context': '지난해 3분기 임금근로 일자리가 지난해 같은 기간보다 36...,None,경제


In [11]:
df['doc_class.code'].unique()

array(['국제', '정치', 'IT과학', '사회', '경제', '문화', '스포츠', '지역', '기타'],
      dtype=object)

In [12]:
use_classes = ['정치', '경제', '사회', '문화', '스포츠', 'IT과학', '국제']

In [13]:
df2 = df[(df['doc_class.code'] != '지역') & (df['doc_class.code'] != '기타')][['doc_id', 'doc_title', 'doc_class.code']]
df2.shape

(111998, 3)

In [14]:
df2.drop_duplicates()

df2.shape

(111998, 3)

In [15]:
df2['doc_class.code'].unique()

array(['국제', '정치', 'IT과학', '사회', '경제', '문화', '스포츠'], dtype=object)

In [16]:
df2['doc_class.code'] = df2['doc_class.code'].replace('국제', '세계')
df2['doc_class.code'] = df2['doc_class.code'].replace('문화', '생활문화')

df2['doc_class.code'].unique()

array(['세계', '정치', 'IT과학', '사회', '경제', '생활문화', '스포츠'], dtype=object)

In [17]:
train_df.columns, df2.columns

(Index(['ID', 'input_text', 'label_text', 'target', 'predefined_news_category',
        'annotations', 'url', 'date'],
       dtype='object'),
 Index(['doc_id', 'doc_title', 'doc_class.code'], dtype='object'))

In [18]:
label2num = {
    label: i for i, label in enumerate(['정치', '경제', '사회', '생활문화', '세계', 'IT과학', '스포츠'])
}
label2num

{'정치': 0, '경제': 1, '사회': 2, '생활문화': 3, '세계': 4, 'IT과학': 5, '스포츠': 6}

In [19]:
df2.columns = ['ID', 'input_text', 'label_text']
df2['target'] = df2['label_text'].apply(lambda x: label2num[x])

df2.columns

Index(['ID', 'input_text', 'label_text', 'target'], dtype='object')

In [20]:
print(pd.concat([train_df, df2], axis=0).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text']).shape)
print(pd.concat([train_df, df2], axis=0).drop_duplicates(subset=['input_text', 'target']).shape)

(164528, 8)
(161537, 8)
(162937, 8)


In [21]:
# df2.to_csv('../data/ai_hub_017.csv', index=False)

# 3. AI HUB 낚시 기사성 뉴스 데이터

In [22]:
ai_hub_146_val_path_list = glob("../data/ai_hub_146_val/*/*.json")
len(ai_hub_146_val_path_list)

46903

In [23]:
id = []
category = []
title = []

for path in ai_hub_146_val_path_list:
    new_df = pd.read_json(path)

    id.append(new_df.loc['newsID']['sourceDataInfo'])
    category.append(new_df.loc['newsCategory']['sourceDataInfo'])
    title.append(new_df.loc['newsTitle']['sourceDataInfo'])

df = pd.DataFrame()
df['ID'] = id
df['input_text'] = category
df['label_text'] = title

df.shape

(46903, 3)

In [24]:
df.columns = ['ID', 'label_text', 'input_text']
df['label_text'].unique()

array(['IT&과학', '세계', '정치', '사회', '생활&문화', '경제'], dtype=object)

In [25]:
df['label_text'] = df['label_text'].replace(r'&', '', regex=True)
df['label_text'].unique()

array(['IT과학', '세계', '정치', '사회', '생활문화', '경제'], dtype=object)

In [26]:
df2['target'] = df2['label_text'].apply(lambda x: label2num[x])
df2.head()

,ID,input_text,label_text,target
0,01100201.20210225173715002,"“시진핑엔 선물, 바이든엔 모욕”… 유럽의회서 막힌 中·EU투자협정",세계,4
1,01100201.20210225092831001,김경수 분노…“내가 문 대통령께 반발? 참 희한한 일”,정치,0
2,01100201.20210225111004001,오늘부터 사라진 네이버 ‘실검’…빈자리엔 날씨와 주가,IT과학,5
3,01100201.20210225040953001,"경남도, 신혼 반값 임대주택 공급 확대",사회,2
4,01100201.20210225161345001,작년 3분기 일자리 늘었다는데… 2030 일자리만 15만개 감소,경제,1


In [27]:
# df2.to_csv('../data/ai_hub_146_val.csv', index=False)

# 4. AI HUB 데이터끼리 합치기

In [28]:
ai_hub_017 = pd.read_csv('../data/ai_hub_017.csv')
ai_hub_146 = pd.read_csv('../data/ai_hub_146_val.csv')
ai_hub = pd.concat([ai_hub_017, ai_hub_146], axis=0)

ai_hub.shape

(223996, 4)

In [50]:
ai_hub.drop_duplicates(subset=['input_text', 'target'], inplace=True)
ai_hub.shape

(110563, 4)

In [55]:
ai_hub[ai_hub.duplicated(subset=['input_text'], keep=False)].sort_values(by='input_text')

,ID,input_text,label_text,target
29288,JAI_article_0820.1542,"""1984년부터 틀린적 없다"" 대선승자 아는 美증시···이번엔?",세계,4
29415,JAI_article_0820.1669,"""1984년부터 틀린적 없다"" 대선승자 아는 美증시···이번엔?",경제,1
29764,JAI_article_0820.2018,"""가르치는 일이 나 자신"" 사상 첫 직업 가진 퍼스트레이디 탄생",경제,1
29533,JAI_article_0820.1787,"""가르치는 일이 나 자신"" 사상 첫 직업 가진 퍼스트레이디 탄생",세계,4
29413,JAI_article_0820.1667,"""경제 좋으면 재선 실패 없다"" 美대선 공식, 트럼프도 통할까",경제,1
29406,JAI_article_0820.1660,"""경제 좋으면 재선 실패 없다"" 美대선 공식, 트럼프도 통할까",세계,4
29715,JAI_article_0820.1969,"4년 전과 정반대 러스트벨트 표심…샤이 트럼프, 히든 바이든에 졌다",세계,4
29750,JAI_article_0820.2004,"4년 전과 정반대 러스트벨트 표심…샤이 트럼프, 히든 바이든에 졌다",경제,1
29421,JAI_article_0820.1675,"NYT ""韓정부는 트럼프, 국민은 바이든 원해""…각국 복잡 셈법",세계,4
29442,JAI_article_0820.1696,"NYT ""韓정부는 트럼프, 국민은 바이든 원해""…각국 복잡 셈법",경제,1


In [56]:
del_duplicate_index = [29288, 29764, 29406, 29750, 29442, 29773, 54108, 29626, 29450, 10340, 25707, 42812, 11774, 29751,
                       29443, 30159, 30137, 29813, 24469, 40823, 29762, 82929, 79340, 29810, 30135, 29801, 29752, 18799, 30136]

ai_hub.drop(del_duplicate_index, axis=0, inplace=True)
ai_hub.shape

(110534, 4)

In [57]:
ai_hub['target'].value_counts()

target
2    39873
1    19867
3    19844
0    14164
6     6674
4     5470
5     4642
Name: count, dtype: int64

In [60]:
test_count = [722, 1348, 3701, 1369, 835, 554, 578]

for idx, count in enumerate(test_count):
    print(idx, ">", round(count / 9107 * 100, 1))

0 > 7.9
1 > 14.8
2 > 40.6
3 > 15.0
4 > 9.2
5 > 6.1
6 > 6.3


In [63]:
(ai_hub['target'].value_counts(normalize=True) * 100).sort_index()

target
0    12.814157
1    17.973655
2    36.073063
3    17.952847
4     4.948704
5     4.199613
6     6.037961
Name: proportion, dtype: float64

In [82]:
# ai_hub.to_csv('../data/ai_hub_original.csv', index=False)

In [ ]:
# ai_hub 분포 맞추기

# 5. 최종 데이터셋 생성

In [77]:
train_no_g2p = pd.read_csv('../data/train_No_G2P.csv')
label_error_detecting = pd.read_csv('../data/label_error_detecting.csv')

train_no_g2p.shape, label_error_detecting.shape

((47049, 8), (386, 10))

In [78]:
label_error_detecting.columns

Index(['Index', 'ID', 'input_text', 'label_text', '기범', '희진', '주형', '소영', '재원',
       '정답'],
      dtype='object')

In [81]:
label_error_detecting.tail()

,Index,ID,input_text,label_text,기범,희진,주형,소영,재원,정답
381,381,ynat-v1_train_44737,中경제성장 양극화 심화…상반기 랴오닝성 마이너스 성장,세계,1,1,1,1,1,True
382,382,ynat-v1_train_44784,재상고 포기 이재현 CJ 회장 형집행정지 가능성은,IT과학,0,0,0,0,0,False
383,383,ynat-v1_train_44784,재상고 포기 이재현 CJ 회장 형집행정지 가능성은,사회,1,1,1,1,1,True
384,384,ynat-v1_train_45083,4차산업혁명위원회 첫 회의…사람 중심 정책수립 논의,사회,1,1,1,1,1,True
385,385,ynat-v1_train_45083,4차산업혁명위원회 첫 회의…사람 중심 정책수립 논의,세계,0,0,0,0,0,False


In [80]:
label_error_detecting['정답'].value_counts()

정답
False    194
True     192
Name: count, dtype: int64

In [79]:
train_no_g2p.drop(label_error_detecting[label_error_detecting['정답'] == False].index, axis=0, inplace=True)
train_no_g2p.shape, 47049 - 193

((46855, 8), 46851)

In [76]:
train_no_g2p[train_no_g2p.duplicated(subset=['ID', 'input_text'])].shape, 1365 + 198

((1365, 8), 1563)